In [3]:
#microphone check one two what is this!

In [10]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

import re

from thefuzz import process

In [12]:
tube_perf = pd.read_excel("tfl-tube-performance.xlsx")

entry_exit = pd.read_excel("multi-year-station-entry-and-exit-figures.xlsx", sheet_name=None)

df_list = []
for year, data in entry_exit.items():
    data['year'] = year
    df_list.append(data)
entry_exit_df = pd.concat(df_list, ignore_index=True)

In [13]:
df_list = []
for sheet, data in entry_exit.items():
    year = re.findall(r'\d{4}', sheet)
    data['year'] = int(year[0]) if year else None
    df_list.append(data)

entry_exit_df = pd.concat(df_list, ignore_index=True)

In [14]:
tube_perf.columns = tube_perf.columns.str.strip().str.lower().str.replace(" ", "_")
entry_exit_df.columns = entry_exit_df.columns.str.strip().str.lower().str.replace(" ", "_")

In [15]:
tube_perf = tube_perf[['name', 'london_underground_performance_data']].rename(columns={'name': 'station_name'})
entry_exit_df = entry_exit_df[['station', 'local_authority', 'zone(s)[†]', 'year', 'usage[5]']]

#fuzzy matching
station_names_perf = tube_perf['station_name'].dropna().unique()
entry_exit_df['matched_station'] = entry_exit_df['station'].apply(
    lambda x: process.extractOne(x, station_names_perf)[0] if isinstance(x, str) else None)

In [16]:
#merging datasets
merged_df = pd.merge(
    entry_exit_df,
    tube_perf,
    left_on='matched_station',
    right_on='station_name',
    how='left')

print("Merged shape:", merged_df.shape)
print(merged_df.head())

Merged shape: (196718, 8)
  station local_authority zone(s)[†]  year usage[5] matched_station  \
0     NaN             NaN        NaN  None      NaN            None   
1     NaN             NaN        NaN  None      NaN            None   
2     NaN             NaN        NaN  None      NaN            None   
3     NaN             NaN        NaN  None      NaN            None   
4     NaN             NaN        NaN  None      NaN            None   

  station_name                london_underground_performance_data  
0          NaN                                                NaN  
1          NaN                                                NaN  
2          NaN                                                NaN  
3          NaN                                                NaN  
4          NaN  The analysis of each period are taken from the...  
